In [210]:
from util import *
from estimator import *
import numpy as np
import random
import tensorflow as tf
slim = tf.contrib.slim
tf.reset_default_graph()

In [211]:
eps = 1e-7
ns  = 10
batch = 8070
tau = tf.placeholder(tf.float32)
cc  = tf.placeholder(tf.float32,[200])
dd  = tf.placeholder(tf.float32)
kk  = tf.placeholder(tf.float32,[200])

lr  = tf.placeholder(tf.float32)
y_ = tf.placeholder(tf.float32, [batch, 392])
x  = tf.placeholder(tf.float32, [batch, 392])

In [212]:
def nets(estimator):
    h = slim.fully_connected(x,200,activation_fn=tf.nn.sigmoid)
    nh= tf.tile(h,[ns,1])
    u = tf.random_uniform(tf.shape(nh))
    s = tf.reshape(estimator(nh,u,tau,cc,dd,kk), [-1,200])
    h2= slim.fully_connected(slim.flatten(s),200,activation_fn=tf.nn.sigmoid)
    u2= tf.random_uniform(tf.shape(h2))
    s2= tf.reshape(estimator(h2,u2,tau,cc,dd,kk), [-1,200])
    y = slim.fully_connected(s2,392,activation_fn=None)
    
    yy  = tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(labels=tf.tile(y_,[ns,1]),logits=y),1)
    ye  = -tf.reduce_logsumexp(tf.reshape(-yy,[ns,-1,1]), 0) + tf.log(tf.cast(ns,tf.float32))

    return tf.reduce_mean(ye), ye, tf.reduce_mean(tf.reshape(tf.nn.sigmoid(y),[ns,-1,392]),0)

In [213]:
def fit_model(filename,_lr,t,c,k):
    steps = 100
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        saver = tf.train.Saver()
        train_loss = np.empty(steps)
        val_loss   = np.empty(steps) 
        test_loss  = np.empty(steps)
        var_g      = np.empty((steps))        
        
        d = 2.
        for i in xrange(steps*1000):
            batch_   = np.reshape(random.sample(data_train, batch), [batch,-1])
            batch_xs = (np.random.uniform(0.,1.,[batch,392])<batch_[:, 0:392]).astype(float)
            batch_ys = (np.random.uniform(0.,1.,[batch,392])<batch_[:, 392:784]).astype(float)
            res, _ = sess.run([loss, train], {x: batch_xs, y_: batch_ys, lr: _lr, tau:t, cc:c, kk:k, dd:d})
            
            if i % 1000 == 1:
                train_loss[i/1000] = res
                var_g[i/1000] = sess.run(vg,{x:batch_xs, y_:batch_ys, lr:_lr, tau:t, cc:c, kk:k, dd:d})
                
                if filename[:2]=='MX':
                    d = 3. - np.exp(-0.00001*i)
                    c = np.exp(-0.00001*i)
                    k = 1. - np.minimum(np.exp(-0.00001*i),0.1)
                else:
                    t = np.maximum(np.exp(-0.00003*i),0.1)
                    
                batch_ = np.reshape(random.sample(data_val, batch), [batch,-1])
                batch_xs, batch_ys = batch_[:, 0:392], batch_[:, 392:784]
                val_loss[i/1000] = sess.run(loss, {x: batch_xs, y_: batch_ys, tau:1e-3, cc:1., kk:1.,dd:5.})

                batch_ = np.reshape(random.sample(data_test, batch), [batch,-1])
                batch_xs, batch_ys = batch_[:, 0:392], batch_[:, 392:784]
                test_loss[i/1000] = sess.run(loss, {x: batch_xs, y_: batch_ys, tau:1e-3, cc:1., kk:1.,dd:5.})
                print train_loss[i/1000],test_loss[i/1000],var_g[i/1000]
        np.save('thesis_data/SNN/OMNIGLOT/'+filename+'/loss_rec', [train_loss, val_loss, test_loss, var_g])
        save_path = saver.save(sess, 'thesis_data/SNN/OMNIGLOT/'+filename+"/model.ckpt")

In [214]:
#data_train, data_val, data_test = bmnist()
data_train, data_val, data_test = bomniglot()

In [87]:
loss, vl, _ = nets(mse)
vg = [jacobian(vl, slim.get_model_variables()[z]) for z in range(4)]
vg = [tf.reduce_sum(tf.square(z-tf.reduce_mean(z,0))) for z in vg]
vg = tf.reduce_mean(vg)
train=tf.train.AdamOptimizer(learning_rate=lr).minimize(loss,var_list=slim.get_model_variables())
fit_model('MX1e-3', 1e-3, .1, 1., .9)

270.129669189 255.919815063 0.131634056568
72.6702346802 69.1760559082 0.0973714739084
83.5995559692 78.8026046753 0.503358900547
68.9828567505 86.7719345093 0.437556624413
64.8835830688 69.2442016602 0.289725899696
74.7145614624 61.2848854065 0.554977595806
77.3778762817 78.6003494263 0.569929540157
72.0613937378 67.7802352905 0.951523721218
62.230682373 69.3622360229 0.725941717625
71.0205001831 67.1907272339 0.599214851856
70.8999557495 79.8214187622 0.657054126263
60.94480896 67.6233139038 0.747613370419
90.2439880371 64.1577682495 1.85720026493
71.4847488403 81.0579452515 0.624397277832
78.3867263794 80.7732086182 1.29942417145
65.3676986694 71.8241424561 0.845887064934
70.1993026733 80.8053894043 0.745782911777
62.8126411438 77.4190139771 0.775200724602
86.5014419556 62.955871582 2.16390895844
59.4989814758 67.3432693481 0.55821365118
81.7664337158 77.074760437 1.47970151901
78.9359817505 78.0291671753 1.36249792576
78.3799743652 72.4192199707 0.941338062286
67.5746536255 66.8059

In [215]:
def gen(dn,mn):
    with tf.Session() as sess:
        saver = tf.train.Saver()
        batch_ = data_test
        batch_xs, batch_ys = batch_[:, 0:392], batch_[:, 392:784]
        s_ = []
        s_.append(batch_ys[:100])
        for ii in mn:
            saver.restore(sess, dn+ii+"/model.ckpt")

            tl, s = sess.run([tnll,ss], {x: batch_xs, y_: batch_ys})
            print tl
            s_.append(s[:100])
        return batch_xs[:100], s_

In [208]:
tnll, _, ss = nets(discrete)
a, b = gen('SNN/MNIST/',
           ['GB1e-3','ST3e-4_','AST1e-3','MX1e-3_3.0'])

INFO:tensorflow:Restoring parameters from SNN/MNIST/GB1e-3/model.ckpt
61.0764
INFO:tensorflow:Restoring parameters from SNN/MNIST/ST3e-4_/model.ckpt
61.7474
INFO:tensorflow:Restoring parameters from SNN/MNIST/AST1e-3/model.ckpt
61.5708
INFO:tensorflow:Restoring parameters from SNN/MNIST/MX1e-3_3.0/model.ckpt
61.1707


In [216]:
tnll, _, ss = nets(discrete)
a, b = gen('SNN/OMNIGLOT/',
           ['GB1e-3_','ST3e-4','AST1e-3','MX1e-3_3.0'])

INFO:tensorflow:Restoring parameters from SNN/OMNIGLOT/GB1e-3_/model.ckpt
70.7637
INFO:tensorflow:Restoring parameters from SNN/OMNIGLOT/ST3e-4/model.ckpt
70.431
INFO:tensorflow:Restoring parameters from SNN/OMNIGLOT/AST1e-3/model.ckpt
70.8851
INFO:tensorflow:Restoring parameters from SNN/OMNIGLOT/MX1e-3_3.0/model.ckpt
70.7814


In [217]:
draw(a,b,'62',1)